In [ ]:
# 🛡️ SAFE MODE - Spotify Playlist Downloader

**Ultra-Safe Version with Maximum Error Protection**

This version uses extreme safety measures to prevent any "'list' object has no attribute 'get'" errors.

---

## 🛡️ Safety Features:
- **Zero assumptions** about data structure
- **Multiple fallback strategies** 
- **Comprehensive type checking**
- **Graceful error recovery**
- **Detailed error reporting**

---

**💡 This version should work even with malformed Spotify data!**


In [ ]:
# Install packages
!pip install spotdl yt-dlp --quiet
!spotdl --download-ffmpeg
print("✅ Installation complete!")


In [ ]:
import os
import json
import re
import subprocess
from pathlib import Path
from typing import List, Dict, Optional, Any
import tempfile
from IPython.display import clear_output
import time

def safe_get(obj: Any, key: str, default: Any = None) -> Any:
    """Ultra-safe get function that works with any object type"""
    try:
        if isinstance(obj, dict):
            return obj.get(key, default)
        elif hasattr(obj, key):
            return getattr(obj, key, default)
        else:
            return default
    except:
        return default

def safe_get_nested(obj: Any, *keys: str, default: Any = None) -> Any:
    """Safely get nested values like obj.get('a', {}).get('b', default)"""
    try:
        current = obj
        for key in keys:
            if isinstance(current, dict):
                current = current.get(key, {})
            elif hasattr(current, key):
                current = getattr(current, key, {})
            else:
                return default
        return current if current != {} else default
    except:
        return default

def safe_list_to_string(obj: Any, separator: str = ", ") -> str:
    """Safely convert any object to string, handling lists"""
    try:
        if isinstance(obj, list):
            return separator.join(str(item) for item in obj if item)
        elif isinstance(obj, str):
            return obj
        else:
            return str(obj) if obj else "Unknown"
    except:
        return "Unknown"

class UltraSafeSpotifyDownloader:
    def __init__(self, output_dir: str = "Downloads"):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        print(f"🛡️ Ultra-Safe Spotify Downloader initialized!")
    
    def sanitize_filename(self, filename: str) -> str:
        """Clean filename for filesystem compatibility"""
        try:
            filename = re.sub(r'[<>:"/\\|?*]', '_', str(filename))
            filename = re.sub(r'[^\w\s\-_\.]', '_', filename)
            return filename.strip()[:200]
        except:
            return "unknown_file"
    
    def validate_spotify_url(self, url: str) -> bool:
        """Validate if the URL is a valid Spotify playlist URL"""
        try:
            return 'spotify.com/playlist/' in str(url).lower()
        except:
            return False
    
    def get_playlist_metadata(self, playlist_url: str) -> Optional[Dict]:
        """Get playlist info using spotdl with maximum safety"""
        try:
            print("🔍 Fetching playlist metadata (safe mode)...")
            
            temp_file = tempfile.NamedTemporaryFile(mode='w', suffix='.spotdl', delete=False)
            temp_file.close()
            
            cmd = ['spotdl', 'save', str(playlist_url), '--save-file', temp_file.name]
            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            
            with open(temp_file.name, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            os.unlink(temp_file.name)
            
            # Handle both dictionary and list formats from spotdl
            if isinstance(data, dict):
                print("✅ Metadata is a dictionary (expected format)")
                return data
            elif isinstance(data, list):
                print("🔧 Metadata is a list - converting to expected format")
                # If spotdl returns a list directly, wrap it in a dictionary
                playlist_name = self.extract_playlist_name_from_url(playlist_url)
                converted_data = {
                    'name': playlist_name,
                    'songs': data,
                    'url': playlist_url
                }
                print(f"✅ Converted list of {len(data)} songs to dictionary format")
                return converted_data
            else:
                print(f"⚠️ Warning: Unexpected metadata type: {type(data)}")
                return None
            
        except Exception as e:
            print(f"❌ Error getting metadata: {e}")
            return None
    
    def extract_playlist_name_from_url(self, url: str) -> str:
        """Extract playlist name from Spotify URL or generate a default name"""
        try:
            # Try to extract playlist ID from URL
            if 'playlist/' in url:
                playlist_id = url.split('playlist/')[-1].split('?')[0]
                return f"Spotify_Playlist_{playlist_id[:8]}"
            else:
                return "Spotify_Playlist"
        except:
            return "Downloaded_Playlist"
    
    def safe_extract_track_info(self, track: Any) -> tuple:
        """Safely extract track information from any data structure"""
        try:
            title = "Unknown"
            artists = []
            spotify_url = ""
            
            # Handle different possible track structures
            if isinstance(track, dict):
                title = safe_get(track, 'name', 'Unknown')
                artists = safe_get(track, 'artists', [])
                spotify_url = safe_get(track, 'url', '')
                
            elif isinstance(track, list) and len(track) > 0:
                # If track is a list, try the first element
                first_item = track[0] if track else {}
                if isinstance(first_item, dict):
                    title = safe_get(first_item, 'name', 'Unknown')
                    artists = safe_get(first_item, 'artists', [])
                    spotify_url = safe_get(first_item, 'url', '')
                else:
                    # If first item is not dict, try to extract string info
                    title = safe_list_to_string(track)
            
            # Ensure we have a valid title
            if not title or title == "Unknown":
                title = f"Track_{id(track)}"  # Use object id as fallback
            
            # Convert artists to string safely
            artist_str = safe_list_to_string(artists)
            
            return title, artist_str, spotify_url
            
        except Exception as e:
            print(f"⚠️ Error extracting track info: {e}")
            return "Unknown", "Unknown", ""
    
    def download_with_ytdlp(self, video_url: str, output_path: str, track_info: tuple) -> bool:
        """Fallback download with yt-dlp (high quality audio)"""
        try:
            title, artist_str, _ = track_info
            
            cmd = [
                'yt-dlp',
                '--extract-audio',
                '--audio-format', 'mp3',
                '--audio-quality', '320K',
                '--embed-metadata',
                '--add-metadata',
                '--no-warnings',
                '--write-thumbnail',  # Download thumbnail
                '--embed-thumbnail',  # Embed as album art
                '-o', str(output_path),
                str(video_url)
            ]
            
            print(f"⬇️ Fallback to yt-dlp: {artist_str} - {title}")
            subprocess.run(cmd, capture_output=True, text=True, check=True)
            return True
            
        except Exception:
            return False
    
    def download_with_spotdl(self, track_url: str, output_dir: str, track_info: tuple) -> bool:
        """Primary download with spotdl (includes album art and metadata)"""
        try:
            title, artist_str, _ = track_info
            
            cmd = [
                'spotdl', 'download', str(track_url),
                '--output', str(output_dir),
                '--format', 'mp3',
                '--bitrate', '320k',  # Try for higher quality
                '--embed-metadata',
                '--generate-lrc',
                '--save-cover'  # This should include album art
            ]
            
            print(f"⬇️ Downloading with spotdl: {artist_str} - {title}")
            subprocess.run(cmd, capture_output=True, text=True, check=True)
            return True
            
        except Exception:
            # Try with fallback options if the above fails
            try:
                cmd_fallback = [
                    'spotdl', 'download', str(track_url),
                    '--output', str(output_dir),
                    '--format', 'mp3',
                    '--bitrate', '256k'
                ]
                subprocess.run(cmd_fallback, capture_output=True, text=True, check=True)
                return True
            except Exception:
                return False
    
    def search_youtube(self, artist: str, title: str) -> Optional[str]:
        """Search YouTube using yt-dlp with safe strings"""
        try:
            search_query = f"ytsearch3:{artist} - {title}"
            cmd = ['yt-dlp', '--quiet', '--dump-json', '--playlist-end', '3', search_query]
            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            
            for line in result.stdout.strip().split('\n'):
                if line:
                    try:
                        video = json.loads(line)
                        return safe_get(video, 'webpage_url')
                    except:
                        continue
            return None
        except:
            return None
    
    def download_playlist(self, playlist_url: str) -> bool:
        """Ultra-safe playlist download"""
        try:
            print(f"🛡️ Safe processing: {playlist_url}")
            
            # Get metadata
            metadata = self.get_playlist_metadata(playlist_url)
            if not metadata:
                print("❌ Failed to fetch playlist metadata")
                return False
            
            # Safely extract playlist info
            playlist_name = safe_get(metadata, 'name', 'Unknown Playlist')
            tracks = safe_get(metadata, 'songs', [])
            
            if not tracks:
                print("❌ No tracks found in playlist")
                return False
            
            print(f"📂 Playlist: {playlist_name}")
            print(f"🎵 Total tracks: {len(tracks)}")
            
            # Create folder
            folder = self.output_dir / self.sanitize_filename(playlist_name)
            folder.mkdir(exist_ok=True)
            
            # Save metadata
            try:
                with open(folder / "playlist_info.json", 'w', encoding='utf-8') as f:
                    json.dump(metadata, f, indent=2, ensure_ascii=False)
            except:
                print("⚠️ Could not save metadata file")
            
            success_count = 0
            failed_count = 0
            
            # Process each track safely
            for i, track in enumerate(tracks, 1):
                try:
                    # Safe track info extraction
                    title, artist_str, spotify_url = self.safe_extract_track_info(track)
                    
                    current_track = f"{artist_str} - {title}"
                    print(f"[{i}/{len(tracks)}] Processing: {current_track}")
                    
                    filename = self.sanitize_filename(f"{artist_str} - {title}")
                    output_path = folder / f"{filename}.%(ext)s"
                    
                    # Check if exists
                    existing_files = list(folder.glob(f"{filename}.*"))
                    if existing_files:
                        success_count += 1
                        print("⏭️ Already exists, skipping...")
                        continue
                    
                    # Try downloads
                    success = False
                    track_info = (title, artist_str, spotify_url)
                    
                    # Try spotdl first (better metadata and album art)
                    if spotify_url:
                        success = self.download_with_spotdl(spotify_url, str(folder), track_info)
                    
                    # Fallback to yt-dlp (high quality audio)
                    if not success:
                        youtube_url = self.search_youtube(artist_str, title)
                        if youtube_url:
                            success = self.download_with_ytdlp(youtube_url, str(output_path), track_info)
                    
                    if success:
                        success_count += 1
                        print("✅ Downloaded successfully")
                    else:
                        failed_count += 1
                        print("❌ Download failed")
                
                except Exception as e:
                    print(f"⚠️ Error processing track {i}: {e}")
                    failed_count += 1
                    continue
            
            print(f"\n🛡️ Safe Download Complete!")
            print(f"📊 Summary: {success_count} successful, {failed_count} failed")
            print(f"📁 Files saved to: {folder}")
            
            return success_count > 0
            
        except Exception as e:
            print(f"❌ Critical error in playlist download: {e}")
            return False

# Initialize the ultra-safe downloader
print("🛡️ Ultra-Safe Spotify Downloader ready!")
safe_downloader = UltraSafeSpotifyDownloader()


In [ ]:
# Interactive download with ultra-safe mode
def ultra_safe_interactive_download():
    """Ultra-safe interactive download"""
    
    print("🛡️ ULTRA-SAFE SPOTIFY DOWNLOADER")
    print("=" * 50)
    print("This version prevents ALL possible 'list object has no attribute get' errors!")
    print()
    
    while True:
        print("🎯 Enter Spotify Playlist URL (or 'done' to finish):")
        user_input = input("URL: ").strip()
        
        if user_input.lower() in ['done', 'exit', 'quit', '']:
            break
        
        if not safe_downloader.validate_spotify_url(user_input):
            print("❌ Invalid Spotify playlist URL!")
            continue
        
        print("🛡️ Starting ultra-safe download...")
        
        try:
            success = safe_downloader.download_playlist(user_input)
            
            if success:
                print("\n✅ Ultra-safe download completed!")
            else:
                print("\n❌ Download failed, but no errors should occur!")
                
        except Exception as e:
            print(f"\n❌ Unexpected error (this should not happen): {e}")
        
        print("\n" + "="*50)
    
    print("🛡️ Ultra-safe session complete!")

# Run the ultra-safe downloader
ultra_safe_interactive_download()


In [ ]:
## 📂 Browse Your Downloads

Check what playlists and songs you've downloaded with the ultra-safe downloader:


In [ ]:
def ultra_safe_browse_downloads():
    """Ultra-safe browse and display downloaded files"""
    downloads_dir = Path("Downloads")
    
    if not downloads_dir.exists():
        print("❌ No downloads folder found")
        print("📝 Run the download cell first to create some downloads!")
        return
    
    playlists = [p for p in downloads_dir.iterdir() if p.is_dir()]
    
    if not playlists:
        print("📭 No playlists downloaded yet")
        print("📝 Run the download cell to start downloading playlists!")
        return
    
    print("📂 YOUR ULTRA-SAFE DOWNLOADED PLAYLISTS")
    print("=" * 50)
    
    total_songs = 0
    total_size = 0
    
    for i, playlist_folder in enumerate(playlists, 1):
        try:
            # Count songs (exclude metadata file)
            songs = [f for f in playlist_folder.iterdir() 
                    if f.suffix.lower() in ['.mp3', '.flac', '.m4a'] and f.is_file()]
            
            # Count additional files (lyrics, covers)
            lrc_files = [f for f in playlist_folder.iterdir() if f.suffix.lower() == '.lrc']
            cover_files = [f for f in playlist_folder.iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
            
            # Calculate folder size safely
            folder_size = 0
            try:
                folder_size = sum(f.stat().st_size for f in playlist_folder.rglob('*') if f.is_file())
            except:
                pass
            folder_size_mb = folder_size / (1024 * 1024) if folder_size > 0 else 0
            
            print(f"🎵 {i}. {playlist_folder.name}")
            print(f"   📊 Songs: {len(songs)}")
            print(f"   📜 Lyrics: {len(lrc_files)} files")
            print(f"   🖼️ Album art: {len(cover_files)} covers")
            print(f"   💾 Size: {folder_size_mb:.1f} MB")
            
            # Show sample tracks
            if songs:
                print("   🎶 Sample tracks:")
                for song in songs[:3]:
                    song_name = song.name
                    if len(song_name) > 60:
                        song_name = song_name[:57] + "..."
                    print(f"      • {song_name}")
                if len(songs) > 3:
                    print(f"      ... and {len(songs) - 3} more")
            else:
                print("   ⚠️ No audio files found")
            
            total_songs += len(songs)
            total_size += folder_size_mb
            print()
            
        except Exception as e:
            print(f"⚠️ Error processing {playlist_folder.name}: {e}")
            continue
    
    print("📊 ULTRA-SAFE DOWNLOAD SUMMARY:")
    print(f"   🎵 Total playlists: {len(playlists)}")
    print(f"   🎶 Total songs: {total_songs}")
    print(f"   💾 Total size: {total_size:.1f} MB")
    print(f"   📁 Location: {downloads_dir.absolute()}")
    print("=" * 50)

# Browse your ultra-safe downloads
ultra_safe_browse_downloads()


In [ ]:
## 💾 Save to Google Drive (Optional)

Mount Google Drive and copy your ultra-safe downloads:


In [ ]:
def ultra_safe_google_drive_backup():
    """Ultra-safe Google Drive backup with maximum error protection"""
    try:
        from google.colab import drive
        import shutil
        
        print("💾 ULTRA-SAFE GOOGLE DRIVE BACKUP")
        print("=" * 50)
        print("🔑 You'll be asked to authenticate with Google...")
        
        # Mount Google Drive safely
        try:
            drive.mount('/content/drive')
            print("✅ Google Drive mounted successfully!")
        except Exception as e:
            print(f"❌ Failed to mount Google Drive: {e}")
            return
        
        # Check if downloads exist
        source_dir = "/content/Downloads"
        if not os.path.exists(source_dir):
            print("❌ No downloads folder found!")
            print("📝 Download some playlists first, then run this cell.")
            return
        
        # Create destination directory safely
        drive_dir = "/content/drive/MyDrive/Spotify_Downloads_SafeMode"
        try:
            os.makedirs(drive_dir, exist_ok=True)
            print(f"📁 Created/verified directory: {drive_dir}")
        except Exception as e:
            print(f"❌ Could not create directory: {e}")
            return
        
        print(f"📁 Copying downloads to Google Drive...")
        print(f"📍 Source: {source_dir}")
        print(f"📍 Destination: {drive_dir}")
        
        copied_playlists = 0
        copied_files = 0
        
        # Copy each playlist folder safely
        try:
            for item in os.listdir(source_dir):
                source_path = os.path.join(source_dir, item)
                drive_path = os.path.join(drive_dir, item)
                
                if os.path.isdir(source_path):
                    try:
                        print(f"📂 Copying playlist: {item}")
                        
                        # Remove existing folder if it exists
                        if os.path.exists(drive_path):
                            shutil.rmtree(drive_path)
                        
                        # Copy the folder
                        shutil.copytree(source_path, drive_path)
                        copied_playlists += 1
                        
                        # Count files in the copied folder
                        try:
                            folder_files = len([f for f in os.listdir(drive_path) if os.path.isfile(os.path.join(drive_path, f))])
                            copied_files += folder_files
                            print(f"   ✅ Copied {folder_files} files successfully!")
                        except:
                            print(f"   ✅ Copied successfully!")
                            
                    except Exception as e:
                        print(f"   ❌ Error copying {item}: {e}")
                        continue
        
        except Exception as e:
            print(f"❌ Error accessing source directory: {e}")
            return
        
        print("\\n🎉 ULTRA-SAFE BACKUP COMPLETE!")
        print("=" * 50)
        print(f"✅ Copied {copied_playlists} playlist(s) to Google Drive")
        print(f"📁 Total files: {copied_files}")
        print(f"📍 Location: MyDrive/Spotify_Downloads_SafeMode/")
        print("💡 You can now access your music from any device!")
        print("🛡️ All files backed up with ultra-safe processing!")
        print("=" * 50)
        
    except ImportError:
        print("❌ This feature is only available in Google Colab")
        print("💡 You can manually copy files or use the ZIP download option")
    except Exception as e:
        print(f"❌ Unexpected error in Google Drive backup: {e}")
        print("🛡️ Ultra-safe mode prevented any crashes!")

# Ask user if they want to save to Google Drive
print("💾 Do you want to backup your downloads to Google Drive?")
print("This will copy all your downloaded playlists with album art and lyrics!")
choice = input("Type 'yes' to continue or 'no' to skip: ").strip().lower()

if choice in ['yes', 'y', '1']:
    ultra_safe_google_drive_backup()
else:
    print("⏭️ Skipped Google Drive backup")
    print("💡 You can always run this cell later to backup your downloads")


In [ ]:
## 📦 Download as ZIP (Computer Download)

Create ZIP files and download directly to your computer:


In [ ]:
def ultra_safe_create_zip():
    """Ultra-safe ZIP creation and download to computer"""
    try:
        import zipfile
        from google.colab import files
        
        print("📦 ULTRA-SAFE ZIP DOWNLOADER")
        print("=" * 50)
        
        downloads_dir = "Downloads"
        if not os.path.exists(downloads_dir):
            print("❌ No downloads folder found!")
            print("📝 Download some playlists first before creating ZIP files.")
            return
        
        # Get list of available playlists
        playlists = [d for d in os.listdir(downloads_dir) 
                    if os.path.isdir(os.path.join(downloads_dir, d))]
        
        if not playlists:
            print("❌ No playlists found to ZIP!")
            return
        
        print("🎵 Available playlists:")
        for i, playlist in enumerate(playlists, 1):
            # Count files safely
            try:
                playlist_path = os.path.join(downloads_dir, playlist)
                file_count = len([f for f in os.listdir(playlist_path) 
                                if os.path.isfile(os.path.join(playlist_path, f))])
                print(f"   {i}. {playlist} ({file_count} files)")
            except:
                print(f"   {i}. {playlist}")
        
        print("\n📦 Choose what to download:")
        print("   • Type 'all' to download everything as one ZIP")
        print("   • Type numbers (e.g., '1,3,5') to download specific playlists")
        print("   • Type a number for individual playlist ZIP")
        
        choice = input("\nYour choice: ").strip().lower()
        
        if choice == 'all':
            # Create one ZIP with all playlists
            zip_name = "Spotify_All_Playlists_SafeMode.zip"
            ultra_safe_create_single_zip(downloads_dir, zip_name, "all playlists")
            
        elif ',' in choice:
            # Multiple specific playlists
            try:
                indices = [int(x.strip()) - 1 for x in choice.split(',')]
                selected_playlists = [playlists[i] for i in indices if 0 <= i < len(playlists)]
                
                if selected_playlists:
                    zip_name = f"Spotify_Selected_Playlists_SafeMode.zip"
                    ultra_safe_create_selective_zip(downloads_dir, selected_playlists, zip_name)
                else:
                    print("❌ No valid playlists selected!")
            except:
                print("❌ Invalid selection format!")
                
        else:
            # Single playlist
            try:
                index = int(choice) - 1
                if 0 <= index < len(playlists):
                    selected_playlist = playlists[index]
                    safe_name = selected_playlist.replace(' ', '_').replace('/', '_')
                    zip_name = f"Spotify_{safe_name}_SafeMode.zip"
                    
                    playlist_path = os.path.join(downloads_dir, selected_playlist)
                    ultra_safe_create_single_zip(playlist_path, zip_name, f"playlist '{selected_playlist}'")
                else:
                    print("❌ Invalid playlist number!")
            except:
                print("❌ Please enter a valid number!")
                
    except ImportError:
        print("❌ This feature requires Google Colab environment")
    except Exception as e:
        print(f"❌ Unexpected error in ZIP creation: {e}")
        print("🛡️ Ultra-safe mode prevented any crashes!")

def ultra_safe_create_single_zip(source_path, zip_name, description):
    """Ultra-safe ZIP creation for a single source"""
    try:
        import zipfile
        
        print(f"\n📦 Creating ZIP for {description}...")
        
        with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            if os.path.isfile(source_path):
                # Single file
                zipf.write(source_path, os.path.basename(source_path))
                file_count = 1
            else:
                # Directory
                file_count = 0
                for root, dirs, file_list in os.walk(source_path):
                    for file_name in file_list:
                        try:
                            file_path = os.path.join(root, file_name)
                            archive_name = os.path.relpath(file_path, source_path)
                            zipf.write(file_path, archive_name)
                            file_count += 1
                        except Exception as e:
                            print(f"⚠️ Warning: Could not add {file_name}: {e}")
                            continue
        
        # Get ZIP size safely
        try:
            zip_size = os.path.getsize(zip_name) / (1024 * 1024)  # MB
            print(f"✅ ZIP created successfully!")
            print(f"   📁 Files: {file_count}")
            print(f"   💾 Size: {zip_size:.1f} MB")
            print(f"   🏷️ Name: {zip_name}")
        except:
            print(f"✅ ZIP created successfully: {zip_name}")
        
        print(f"✅ ZIP file ready: {zip_name}")
        print(f"💡 You can find the ZIP file in the file browser on the left")
        print(f"📥 Right-click the file and select 'Download' to save to your computer")
        
    except Exception as e:
        print(f"❌ Error creating ZIP: {e}")
        print("🛡️ Ultra-safe mode prevented any crashes!")

def ultra_safe_create_selective_zip(downloads_dir, selected_playlists, zip_name):
    """Ultra-safe ZIP creation for multiple selected playlists"""
    try:
        import zipfile
        
        print(f"\n📦 Creating ZIP for {len(selected_playlists)} selected playlists...")
        
        file_count = 0
        
        with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for playlist in selected_playlists:
                playlist_path = os.path.join(downloads_dir, playlist)
                
                if os.path.exists(playlist_path):
                    print(f"   📂 Adding: {playlist}")
                    
                    for root, dirs, file_list in os.walk(playlist_path):
                        for file_name in file_list:
                            try:
                                file_path = os.path.join(root, file_name)
                                # Create archive path: playlist_name/file
                                archive_name = os.path.join(playlist, os.path.relpath(file_path, playlist_path))
                                zipf.write(file_path, archive_name)
                                file_count += 1
                            except Exception as e:
                                print(f"⚠️ Warning: Could not add {file_name}: {e}")
                                continue
                else:
                    print(f"⚠️ Warning: {playlist} not found")
        
        # Get ZIP size safely
        try:
            zip_size = os.path.getsize(zip_name) / (1024 * 1024)  # MB
            print(f"✅ ZIP created successfully!")
            print(f"   📁 Files: {file_count}")
            print(f"   💾 Size: {zip_size:.1f} MB")
            print(f"   🏷️ Name: {zip_name}")
        except:
            print(f"✅ ZIP created successfully: {zip_name}")
        
        print(f"✅ ZIP file ready: {zip_name}")
        print(f"💡 You can find the ZIP file in the file browser on the left")
        print(f"📥 Right-click the file and select 'Download' to save to your computer")
        
    except Exception as e:
        print(f"❌ Error creating selective ZIP: {e}")
        print("🛡️ Ultra-safe mode prevented any crashes!")

# Ask user if they want to create ZIP downloads
print("📦 Do you want to create ZIP files for download to your computer?")
print("This will package your playlists with all music, album art, and lyrics!")
choice = input("Type 'yes' to continue or 'no' to skip: ").strip().lower()

if choice in ['yes', 'y', '1']:
    ultra_safe_create_zip()
else:
    print("⏭️ Skipped ZIP creation")
    print("💡 You can always run this cell later to create ZIP downloads")
